<a href="https://colab.research.google.com/github/Ndthain25092002/Movies_RS/blob/main/ContentBase_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F626%2F1187%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240131%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240131T132731Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db6e3428660ff0cc22d83838c0c2a230caead9bcaa2eda34ea4410aa7bd46779351436f29e0f55cee5c55edfa1f00c09578f92feb4048a881c19bd829a544eb231d51e9f30c12841700d987b28b3a0059f2a1756044a4c9825d1614b1254fe9190c489ad28fba79e05b6999efd86c084975767d40a867c3827442d807f4a859e7d915b8e157291d15066557ea757182a1fe0b101bee50f45ca5f883dbaed7c6a744606d967c99d6a82c85e014bd574bcc098e5c03c1758951d9feae47aa28e590f0725c325bd4cd56aa268bc1061d26528484b9e2a7009b5064da3e42cccc5733cc083d63eb0d88946bbe4ad54bb4311a9f555e1698741b21ff44997d6fc0d1b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


Dùng linear regression với input là 1 ma trận feature và score đánh giá

In [ ]:
import pandas as pd

u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../input/ml-100k/u.user', sep='|', names=u_cols,
encoding='latin-1')

n_users = users.shape[0]
print('Number of users:', n_users)


In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('../input/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('../input/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()
print(ratings_base[0:20])

print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])

In [ ]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('../input/ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')

n_items = items.shape[0]
print("Number of items:",n_items)

In [ ]:
X0 = items.as_matrix()
#Thể loại phim ở 19 phần tử cuối X_train_counts là ma trận 1682x19 1682 bộ phim và 19 thể loại phim
X_train_counts = X0[:, -19:]
print(X_train_counts.shape)

Sử dụng hàm TD_IDF để chuyển đổi thành ma trận feature

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()
print(X_train_counts[0])
#Mỗi hàng là các profile của các item (các bộ phim)
print(tfidf[0])

In [ ]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,0] # users
    # Lấy chỉ số đánh giá của user
    # user_id bắt đầu = 1
    ids = np.where(y == user_id +1)[0]
    item_ids = rate_matrix[ids, 1] - 1 # ids - 1
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)
ids, scores = get_items_rated_by_user(rate_train, 0)
list_head = np.concatenate((ids.reshape(len(ids),1), scores.reshape(len(scores),1)),axis = 1)[0:10]
print(list_head)

In [ ]:
from sklearn.linear_model import Ridge # mô hình linear regression với regularization

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]

    clf.fit(Xhat, scores)
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

In [ ]:
#model dự đoán
Yhat = tfidf.dot(W) + b
# print(Yhat[0])

In [ ]:
n = 100
ids, scores = get_items_rated_by_user(rate_test, 0)
Yhat[n, ids]
print('Rated movies ids:', ids )
print('True ratings:', scores)
print('Predicted ratings:', Yhat[ids, n])

In [ ]:
from math import sqrt
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred
        se += (e*e).sum(axis = 0)
        cnt += e.size
    return sqrt(se/cnt)


In [ ]:
print('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

In [ ]:
threshold = 3.5

def evaluate_precision_recall(Yhat, rates, W, b, threshold):
    tp = 0  # true positive
    fp = 0  # false positive
    fn = 0  # false negative

    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]

        relevant_items = ids[scores_truth >= threshold]
        recommended_items = ids[scores_pred >= threshold]

        tp += len(set(relevant_items) & set(recommended_items))
        fp += len(set(recommended_items) - set(relevant_items))
        fn += len(set(relevant_items) - set(recommended_items))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall

precision, recall = evaluate_precision_recall(Yhat, rate_test, W, b, threshold)
print('Precision:', precision)
print('Recall:', recall)
